In [1]:
from dataclasses import dataclass
from typing import Annotated, Any, Callable, TypeVar

from autogen import register_function
from autogen.agentchat import UserProxyAgent, AssistantAgent, ConversableAgent
from autogen.io import IOStream

from fastagency.core.base import ConsoleIO, ChatMessage
from fastagency.core.autogen import AutoGenIOAdapter

from fixtures import openai_gpt4o_mini_llm_config, openai_gpt4o_llm_config

In [ ]:
C = TypeVar("C", bound=Callable[..., Any])

@dataclass
class User:
    ask_question: Callable[[str], str]
    report: Callable[[str], None]
    ask_permission_to_execute: Callable[[C], C]

class AutoGenTeam:
    def __init__(self):
        self._factories = {}

    def factory(self, name: str, description: str) -> Any:
        def decorator(func):
            if name in self._factories:
                raise ValueError(f"Factory with name '{name}' already exists")
            
            self._factories[name] = func, description
            return func
        
        return decorator

    def create(self, name: str, session_id: str, io: ConsoleIO, initial_message: str) -> Any:
        team, _ = self._factories[name]
        iostream = AutoGenIOAdapter(io)

        def ask_user(message: Annotated[str, "message for the user"]) -> str:
            msg = ChatMessage(
                sender="student",
                recepient="user",
                heading="User info needed:",
                body=message
            )
            return io.input(msg)
        
        def report_to_user(message: Annotated[str, "message for the user"]) -> str:
            msg = ChatMessage(
                sender="student",
                recepient="user",
                heading="Report:",
                body=message
            )
            return "ok"
            io.print(msg)
        
        user = User(
            ask_question=ask_user,
            report=report_to_user,
        )
        
        with IOStream.set_default(iostream):
            return team(user=user, initial_message=initial_message)
    

In [ ]:
team = AutoGenTeam()

@team.factory(name="my_team", description="Student and teacher chat")
def run_team(user: User, initial_message: str) -> str:
    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student willing to learn.",
        llm_config=openai_gpt4o_mini_llm_config,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a math teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
    )

    chat_result = student_agent.initiate_chat(
        teacher_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=5,
    )

    return chat_result.summary

result = team.create(name="my_team", session_id="session_id", io=ConsoleIO(), initial_message="What is triangle inequality?")

print(result)

Student_Agent (to Teacher_Agent):


What is triangle inequality?


--------------------------------------------------------------------------------


>>>>>>>> USING AUTO REPLY...

Teacher_Agent (to Student_Agent):


The triangle inequality is a fundamental property of triangles in geometry that states that, for any triangle, the sum of the lengths of any two sides must be greater than the length of the third side. This can be expressed mathematically as follows:

If a triangle has sides of lengths \( a \), \( b \), and \( c \), then the triangle inequality can be stated as:

1. \( a + b > c \)
2. \( a + c > b \)
3. \( b + c > a \)

This means that no one side can be equal to or greater than the sum of the other two sides. The triangle inequality is essential for determining whether three lengths can form a triangle and is also a foundational concept in various areas of mathematics, including analysis and geometry.


----------------------------------------------------------------------

In [ ]:
team = AutoGenTeam()

@team.factory(name="my_team", description="Student and teacher chat")
def run_team(user: User, initial_message: str) -> str:

    def is_termination_msg(msg: str) -> bool:
        return msg["content"] is not None and "TERMINATE" in msg["content"]

    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student willing to learn. Before anything, retrieve exam questions by calling a function. Next, write your proposed answers and engage in dialogue with your teacher. Once you are done, register the final answers by calling a function. Finally, terminate the chat by saying 'TERMINATE'.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )

    register_function(
        user.ask_question,
        caller=student_agent,
        executor=teacher_agent,
        name="retrieve_exam_questions",
        description="Get exam questions",
    )

    register_function(
        user.report,
        caller=student_agent,
        executor=teacher_agent,
        name="write_final_answers",
        description="Write a final answers to exam questions after discussing with the teacher.",
    )

    chat_result = teacher_agent.initiate_chat(
        student_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=10,
    )

    return chat_result.summary

result = team.create(name="my_team", session_id="session_id", io=ConsoleIO(), initial_message="Start learning about Leonardo da Vinci.")

print(result)

Teacher_Agent (to Student_Agent):


Start learning about Leonardo da Vinci.


--------------------------------------------------------------------------------

Student_Agent (to Teacher_Agent):


***** Suggested tool call (call_9asdzNI6ZcT75TeEV1xp6A7V): retrieve_exam_questions *****

Arguments: 
{"message":"Please provide the exam questions related to Leonardo da Vinci."}

****************************************************************************************


--------------------------------------------------------------------------------


>>>>>>>> EXECUTING FUNCTION retrieve_exam_questions...

Teacher_Agent (to Student_Agent):


Teacher_Agent (to Student_Agent):


***** Response from calling tool (call_9asdzNI6ZcT75TeEV1xp6A7V) *****

!. Mona Lisa 2. Innovations

**********************************************************************


--------------------------------------------------------------------------------

Student_Agent (to Teacher_Agent):


The exam questions are as fol